In [1]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import sys
import os
import math
from data import load_NEMSIS_dask, load_NEMSIS_pandas, chain_join

In [2]:
# data_dir: os.PathLike = "/Volumes/Extreme Pro/datasets/NEMSIS/ASCII/"
# data_dir: os.PathLike = "/media/jbajor/Extreme Pro/datasets/NEMSIS/ASCII/"
data_dir: os.PathLike = "/home/jbajor/Dev/datasets/NEMSIS/ASCII/"


nemsis_subset = {
    "ComputedElements":[
        'PcrKey',
        'NasemsoRegion',
        'Urbanicity',
        'ageinyear',
        'EMSDispatchCenterTimeSec',
        'EMSChuteTimeMin',
        'EMSSceneResponseTimeMin',
        'EMSSceneTimeMin',
        'EMSSceneToPatientTimeMin'
    ],
    "FACTPCRADDITIONALRESPONSEMODE":None,
    "FACTPCRADDITIONALSYMPTOM":None,
    "FACTPCRADDITIONALTRANSPORTMODE":None,
    "FACTPCRARRESTCPRPROVIDED":None,
    "FACTPCRARRESTRESUSCITATION":None,
    "FACTPCRARRESTRHYTHMDESTINATION":None,
    "FACTPCRARRESTROSC":None,
    "FACTPCRARRESTWITNESS":None,
    "FACTPCRBARRIERTOCARE":None,
    "FACTPCRMEDICATION":None,
    "FACTPCRPRIMARYIMPRESSION":None,
    "FACTPCRPROCEDURE":None,
    "FACTPCRPROTOCOL":None,
    "FACTPCRSECONDARYIMPRESSION":None,
    "FACTPCRTRAUMACRITERIA":None,
    "FACTPCRWORKRELATEDEXPOSURE":None,
    # "PCRMEDCOMPGROUP":None, # bad key
    # "PCRPROCCOMPGROUP":None, # bad key
    # "PCRVITALECGGROUP":None, # bad key
    "Pub_PCRevents":[
        'PcrKey',
        'eArrest_01',
        'eArrest_02',
        'eArrest_05',
        'eArrest_07',
        'eArrest_11',
        'eArrest_14',
        'eArrest_16',
        'eArrest_18',
        'eDisposition_23',
        'ePayment_01',
        'eResponse_07',
        'eResponse_15',
        'eSituation_01',
        'eSituation_07',
        'eSituation_08',
        'eTimes_01',
        'eTimes_03',
        'eTimes_05',
        'eTimes_06',
        'eTimes_07',
        'eTimes_09',
        'eTimes_11',
        'eTimes_12',
        'eTimes_13'
    ],
}


In [13]:
main = pd.read_csv(os.path.join(data_dir, 'Pub_PCRevents' + ".txt"), sep="~\|~", engine="python", nrows=1000000)

sub = pd.read_csv(os.path.join(data_dir, 'FACTPCRPROCEDURE' + ".txt"), sep="~\|~", engine="python")

: 

: 

In [ ]:
main.set_index("'PcrKey'", inplace=True, append=True)
sub.set_index("'PcrKey'", inplace=True, append=True)

In [ ]:
jtest = main.join(sub, how='left')

_____

In [9]:
proctest = pd.read_csv(os.path.join(data_dir, 'FACTPCRPROCEDURE' + ".txt"), sep="~\|~", engine="python", nrows=50000)
maintest = pd.read_csv(os.path.join(data_dir, 'Pub_PCRevents' + ".txt"), sep="~\|~", engine="python", nrows=50000)

In [11]:
proctest.set_index("'PcrKey'", inplace=True, append=True)
maintest.set_index("'PcrKey'", inplace=True, append=True)

In [12]:
jtest = maintest.join(proctest, how='left')

In [3]:
dataframes = load_NEMSIS_pandas(data_dir=data_dir, load_targets=nemsis_subset, nrows=10000)

In [4]:
dlist = [i for i in dataframes.values()]

In [5]:
dlist[0]

,PcrKey,NasemsoRegion,Urbanicity,ageinyear,EMSDispatchCenterTimeSec,EMSChuteTimeMin,EMSSceneResponseTimeMin,EMSSceneTimeMin,EMSSceneToPatientTimeMin
0,3442467,South,,<NA>,<NA>,<NA>,<NA>,<NA>,NaN
1,9059962,South,Urban,<NA>,<NA>,<NA>,<NA>,<NA>,NaN
2,9778729,West,Rural,90,60,2,13,12,NaN
3,9829108,West,Urban,85,10,4.8333333333,5,60,NaN
4,10094816,South,Urban,82,<NA>,<NA>,<NA>,<NA>,NaN
...,...,...,...,...,...,...,...,...,...
9995,28319680,South,Urban,45,4,0.2,43.3,15.966666667,3.4333333333
9996,28319682,South,Urban,<NA>,<NA>,1.5,<NA>,<NA>,<NA>
9997,28319683,South,Urban,<NA>,0,0,4,<NA>,<NA>
9998,28319684,South,Urban,44,2436,0.2833333333,5.8166666667,<NA>,1.6


In [6]:
joined = chain_join(datadict=dataframes, key='PcrKey', start_table='Pub_PCRevents', type='left')

In [ ]:
joined

In [5]:
# joined.to_pickle('NEMSIS_subset.pkl')